Задание из 2-х частей.
Берем отызывы за лето (из архива с материалами или предыдущего занятия)
1. Учим conv сеть для классификации - выбить auc выше 0.95
2. Предобучаем word2vec и его эмбединга инициализируем сетку, как влияет на качество?

In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt

%matplotlib inline

import warnings 
warnings.filterwarnings("ignore", category=Warning)

## 0. Загрузка и обзор данных

In [2]:
data = pd.read_excel("отзывы за лето.xls")
data.head()

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14


In [3]:
data['Rating'].value_counts()

5    14586
1     2276
4     2138
3      911
2      748
Name: Rating, dtype: int64

In [4]:
data = data[data['Rating'] != 3]
data['target'] = data['Rating'] > 3
data['target'] = data['target'].astype(int)

In [5]:
data['target'].value_counts()

1    16724
0     3024
Name: target, dtype: int64

**Положительные отзывы:**

In [6]:
# data.loc[data['target'] == 1, 'Content'].sample(20, random_state=42)

**Отрицательные отзывы:**

In [7]:
# data.loc[data['target'] == 0, 'Content'].sample(20, random_state=42)

## 1. Обработка текстов

In [8]:
import nltk
from pymorphy2 import MorphAnalyzer

Ограничимся минимальной очисткой текстов - только от точек, запятых и предлогов:

In [9]:
punctuation = r'[,.]'
morpher = MorphAnalyzer()

stop = {'без', 'безо', 'близ', 'в', 'во', 'вместо', 'вне',
        'для', 'до', 'за', 'из', 'изо', 'из-за', 'из-под', 
        'к', 'ко', 'кроме', 'между', 'меж', 'на', 'над', 'надо',
        'о', 'об', 'обо', 'от', 'ото', 'перед', 'передо', 'пред', 'предо',
        'пo', 'под', 'подо', 'при', 'про', 'ради', 'с', 'со',
        'сквозь', 'среди', 'у', 'через', 'чрез'}

Функции для обработки текстов:

In [10]:
def normalize_text(text_list):
    return [morpher.parse(word)[0].normal_form \
            for word in text_list if word not in stop]

def preprocess_text(text):    
    text_list = nltk.word_tokenize(str(text))
    text_list = normalize_text(text_list)
    return text_list

Обработка текстов:

In [11]:
data['Text'] = data['Content'].str.lower()
data['Text'] = data['Text'].str.replace(r'[.,]', ' ')
data['Text'] = [preprocess_text(text_list) for text_list in tqdm(data['Text'])]

100%|███████████████████████████████████| 19748/19748 [01:06<00:00, 294.82it/s]


Датафрейм с обработанными текстами:

In [22]:
data['Text'] = data['Text'].str.join(' ')
data.head(n=10)

,Rating,Content,Date,target,Text
0,5,It just works!,2017-08-14,1,i t j u s t w o r k s !
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14,1,ц е л о е у д о б н о н о й п р и л о ж е ...
2,5,Отлично все,2017-08-14,1,о т л и ч н о в е с ь
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14,1,с т а т ь з а в и с а т ь 1 % р а б о ...
4,5,"Очень удобно, работает быстро.",2017-08-14,1,о ч е н ь у д о б н о р а б о т а т ь б ...
5,5,Всё удобно норм 👍👍👍,2017-08-14,1,в е с ь у д о б н о н о р м а 👍 👍 👍
6,5,Очень удобное приложение.,2017-08-14,1,о ч е н ь у д о б н ы й п р и л о ж е н и е
7,5,Все устраивает,2017-08-14,1,в е с ь у с т р а и в а т ь
8,5,У меня работает все четко. В отличии от банком...,2017-08-14,1,я р а б о т а т ь в е с ь ч ё т к о о ...
9,5,Очень все хорошо👍,2017-08-14,1,о ч е н ь в е с ь х о р о ш о 👍


In [13]:
from sklearn.model_selection import train_test_split

In [14]:
X = data['Text']
y = data['target']

In [15]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.25,
    random_state=42,
    stratify=y
)

In [16]:
train_corpus = " ".join(X_train)
train_corpus = train_corpus.lower()

In [17]:
max_words = 2000
max_len = 40

In [18]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download("punkt")

tokens = word_tokenize(train_corpus)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\TOSHIBA\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [19]:
tokens_filtered = [word for word in tokens if word.isalnum()]

In [20]:
from nltk.probability import FreqDist
dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]

In [21]:
tokens_filtered_top[:10]

['приложение',
 'весь',
 'не',
 'и',
 'очень',
 'удобно',
 'я',
 'работать',
 'удобный',
 'что']

In [23]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

In [24]:
import numpy as np
def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [25]:
X_train_sec = np.asarray([text_to_sequence(text, max_len) for text in X_train], dtype=np.int32)
X_test_sec = np.asarray([text_to_sequence(text, max_len) for text in X_test], dtype=np.int32)

In [26]:
X_train_sec.shape

(14811, 40)

In [27]:
X_train_sec[1]

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0, 13,  8, 11])

In [28]:
from gensim.models import Word2Vec

In [31]:
X_train_tokenized = X_train.apply(word_tokenize)

In [32]:
sentences = X_train_tokenized.tolist()
params = {'size': 128, 'window': 5, 'min_count': 2, 'sg': 1, 'hs': 0, 'negative': 10, 'workers': 32, 'seed': 34}

In [33]:
%%time
model_w2v = Word2Vec(**params)
model_w2v.build_vocab(sentences)
model_w2v.train(sentences,
                total_examples=len(sentences),
                epochs=20)

Wall time: 12.7 s


(1547131, 2324220)

In [36]:
model_w2v.most_similar(positive='удобный', topn=5)

[('лёгкое', 0.7478971481323242),
 ('приятный', 0.7298699617385864),
 ('комфортный', 0.7276144027709961),
 ('полезный', 0.7194794416427612),
 ('хорошоя', 0.7185418605804443)]

In [38]:
model_w2v.most_similar(positive='антивирус', topn=5)

[('инициализация', 0.6743736267089844),
 ('встроить', 0.6433878540992737),
 ('антивирусник', 0.6195533871650696),
 ('долбаный', 0.6135855913162231),
 ('вирус', 0.6067352294921875)]

In [40]:
'ужас' in model_w2v

True

In [41]:
def text2vec(text_list, size=128):
    n_words = len(text_list)
    vec = np.zeros((1, size))
    if n_words == 0:
        return vec
    for word in text_list:
        if word in model_w2v:
            vec += model_w2v[word].reshape((1, size))
        else:
            continue
    return vec / n_words

In [52]:
wordvec_df = X_train_tokenized.apply(text2vec)

In [53]:
wordvec_df = np.concatenate(wordvec_df.values)
wordvec_df.shape

(14811, 128)

In [30]:
import numpy as np
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 
from keras.objectives import categorical_crossentropy
from keras.callbacks import EarlyStopping  

In [31]:
num_classes = 2
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [39]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [40]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['AUC'])

In [41]:
history = model.fit(X_train_sec, y_train,
                    batch_size=10,
                    epochs=1,
                    verbose=1,
                    validation_split=0.1)

1333/1333 [==============================] - 12s 9ms/step - loss: 0.2069 - auc: 0.9735 - val_loss: 0.1802 - val_auc: 0.9798


In [43]:
loss, auc = model.evaluate(X_test_sec, y_test, batch_size=10, verbose=1)
print(loss, auc)

4937/4937 [==============================] - 10s 2ms/step - loss: 0.1789 - auc: 0.9805
0.1788545548915863 0.980502188205719


In [ ]:
model.save_weights('/content/drive/My Drive/model_nlp_weights.h5')